In [ ]:
from google.colab import drive
drive.mount('/content/drive')

 creating a final pipeline to take a new food image as input, passing it through ViT for food recognition, then we apply Mask R-CNN for portion estimation, and finally output the estimated calorie count.


Testing: Run our system on a range of food images to evaluate accuracy and portion estimation consistency. *italicized text*

# **1. Input: Food Image**

Process: The pipeline starts by taking a single 2D food image as input.

Expected Outcome: The system will first recognize the type of food in the image, then estimate its portion size, and finally predict the calories based on the estimated portion size and the food type.

# **Food Recognition Using Vision Transformer (ViT)**

Step 1: Load and Fine-Tune ViT

Task: You will use a pre-trained ViT model for food
 classification.
 The model has been fine-tuned to recognize 251
  different food categories.


Input: A food image from the dataset.


Output: The predicted food class (out of 251 possible classes) is returned, e.g., "Pizza."

In [ ]:
# ViT is fine-tuned and ready for inference
vit_model = load_finetuned_vit()  # Placeholder for loading the fine-tuned ViT model

# Passing the input food image for prediction
predicted_class = vit_model.predict(food_image)  # Returns class index,like 101 for "Pizza"
predicted_class_name = class_dict[predicted_class]  # Converting to readable class name
print(f"Predicted Food Class: {predicted_class_name}")


# **Step 2: Retrieve Calorie Information for the Predicted Class**

Task: Retrieve the manually assigned calorie-per-gram value for the predicted food class.

Example:

Pizza: 2.8 kcal/gram

Salad: 0.5 kcal/gram

In [ ]:
#testin purpose dont compile this

# Manually defined calorie density for each food class (example values)
calories_per_gram = {
    "Pizza": 2.8,
    "Salad": 0.5,
    # sample test purspose we have food_estimation.csv
}

# Get the calorie density for the predicted class
calories_per_gram_for_class = calories_per_gram[predicted_class_name]
print(f"Calories per gram for {predicted_class_name}: {calories_per_gram_for_class} kcal/gram")


# **3. Portion Estimation Using Mask R-CNN**

Step 3: Load and Initialize Mask R-CNN

Task: Perform image segmentation on the input image to isolate the
food portions. Use the Mask R-CNN model to detect and segment the
portions of food within the image.

Input: The same food image as used in ViT.

Output: Segmentation masks showing different portions of the food.

In [ ]:
# Use the Mask R-CNN predictor for portion segmentation
outputs = predictor(food_image)  # Returns masks for food portions

# Visualize segmented portions (optional)
v = Visualizer(food_image[:, :, ::-1])
segmented_output = v.draw_instance_predictions(outputs["instances"].to("cpu")).get_image()[:, :, ::-1]
cv2.imshow("Segmented Food Portions", segmented_output)


# **4. Calorie Estimation**

Step 5: Calculate Total Calories

Task: Multiply the estimated portion size (in grams) by the calories per gram of the predicted food class to estimate the total calories for each food portion.


Example:

For example, if:

The portion size is estimated to be 150 grams,

The predicted food is Pizza, which has 2.8 kcal/gram,

Then the total calories would be:

150 grams x 2.8 kcal/gram = 420 kcal.

In [ ]:
# Calculate total calories for each food portion
for i, mask in enumerate(outputs["instances"].pred_masks.cpu().numpy()):
    portion_size_in_pixels = np.sum(mask)
    portion_size_in_grams = portion_size_in_pixels * pixel_to_gram_ratio

    # Calculate the total calories for this portion
    total_calories = portion_size_in_grams * calories_per_gram_for_class
    print(f"Estimated calories for portion {i+1}: {total_calories:.2f} kcal")


# **5. Output: Total Calorie Estimation**

After calculating the portion sizes and total calories, the system

outputs:
Predicted food class (e.g., Pizza),

Estimated portion size (e.g., 150 grams),

Estimated total calories (e.g., 420 kcal).


This information can be used to provide detailed food and calorie insights to the user.

Predicted Food Class: Pizza
Estimated Portion Size: 150 grams
Estimated Calories: 420 kcal
